In [1]:
from pathlib import Path
import json

all_datasets = """SpeechDetection_LJSpeech
SpeechDetection_LibriSpeechTestClean
SpeechDetection_LibriSpeechTestOther
SpeechTextMatching_LJSpeech
SpeechTextMatching_LibriSpeechTestClean
SpeechTextMatching_LibriSpeechTestOther
SpokenTermDetection_LJSpeech
SpokenTermDetection_LibriSpeechTestClean
SpokenTermDetection_LibriSpeechTestOther
EnhancementDetection_LibrittsTestCleanWham
SarcasmDetection_Mustard
DialogueActPairing_DailyTalk
SpeakerVerification_LibriSpeechTestClean
SpeakerVerification_VCTK
MultiSpeakerDetection_LibriSpeechTestClean
MultiSpeakerDetection_VCTK""".split("\n")


In [3]:
from collections import defaultdict
result_path = Path("/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/imagebind_newdata/imagebind1/results3")
for dataset in all_datasets:
    
    ori_result = json.load((result_path/f"full/{dataset}.json").open())
    ori_acc = ori_result["accuracy"]

    
    results_label = json.load((result_path/f"change_label/full/{dataset}.json").open())
    count_label = defaultdict(dict)
    for result in results_label["predictions"]:
        count_label[result["label"]][result["pred"]] = count_label[result["label"]].get(result["pred"], 0) + 1

    new_label_correct = 0
    new_label_correct += count_label.get("true", {}).get("yes", 0)
    new_label_correct += count_label.get("false", {}).get("no", 0)
    new_label_correct += count_label.get("yes", {}).get("true", 0)
    new_label_correct += count_label.get("no", {}).get("false", 0)
    new_label_acc = new_label_correct / len(results_label["predictions"])
    
    ori_label_correct = 0
    ori_label_correct += count_label.get("true", {}).get("true", 0)
    ori_label_correct += count_label.get("false", {}).get("false", 0)
    ori_label_correct += count_label.get("yes", {}).get("yes", 0)
    ori_label_correct += count_label.get("no", {}).get("no", 0)
    ori_label_acc = ori_label_correct / len(results_label["predictions"])

    results_order = json.load((result_path/f"change_order/full/{dataset}.json").open())
    count_order = defaultdict(dict)
    for result in results_order["predictions"]:
        count_order[result["label"]][result["pred"]] = count_order[result["label"]].get(result["pred"], 0) + 1

    
    order_correct = 0
    order_correct += count_order.get("true", {}).get("true", 0)
    order_correct += count_order.get("false", {}).get("false", 0)
    order_correct += count_order.get("yes", {}).get("yes", 0)
    order_correct += count_order.get("no", {}).get("no", 0)
    order_acc = order_correct / len(results_order["predictions"])

    print(dataset, ";", ori_acc,";", order_acc,";",new_label_acc+ori_label_acc, ";", new_label_acc,";", ori_label_acc, ";", dict(count_order), ";", dict(count_label), sep="")

SpeechDetection_LJSpeech;0.9974809160305343;0.9967175572519084;0.9321374045801527;0.06381679389312978;0.9959541984732825;{'no': {'no': 6546, 'yes': 42}, 'yes': {'yes': 6511, 'no': 1}};{'no': {'false': 5728, 'no': 836, 'true': 23, 'yes': 1}, 'yes': {'true': 6483, 'false': 29}}
SpeechDetection_LibriSpeechTestClean;0.8973282442748092;0.8885496183206106;0.8110687022900763;0.05152671755725191;0.8625954198473282;{'no': {'no': 1119, 'yes': 187}, 'yes': {'yes': 1209, 'no': 105}};{'no': {'false': 1087, 'true': 165, 'no': 49, 'yes': 4, '50': 1}, 'yes': {'true': 1038, 'false': 190, 'yes': 86}}
SpeechDetection_LibriSpeechTestOther;0.8285130996937734;0.834977883633889;0.7601224906430759;0.03504593399115345;0.7951684246342293;{'no': {'no': 1281, 'yes': 183}, 'yes': {'yes': 1173, 'no': 302}};{'no': {'false': 1221, 'true': 185, 'no': 55, 'yes': 3}, 'yes': {'true': 1013, 'false': 411, 'no': 3, 'yes': 48}}
SpeechTextMatching_LJSpeech;0.5140458015267175;0.49755725190839695;0.5020610687022901;7.6335877862

In [15]:
new_label_correct

0

In [12]:
count_order.get("true", dict).get("true", 0)

TypeError: descriptor 'get' for 'dict' objects doesn't apply to a 'str' object

In [80]:
correct = 0
if count.get("true"):
    correct += count["true"].get("yes", 0)
if count.get("false"):
    correct += count["false"].get("no", 0)
if count.get("yes"):
    correct += count["yes"].get("true", 0)
if count.get("no"):
    correct += count["no"].get("false", 0)

for ori_label, v1 in count.items():
    print(ori_label)
    for pred, v2 in v1.items():
        print(pred, v2, end=";")
    print()

print()
print(,";",correct, ";",correct/len(results["predictions"]),";", dict(count))

SyntaxError: invalid syntax (3667233772.py, line 18)

In [1]:
def cal_accuracy(predictions, training_instructions=None):
    seen_count = 0
    unseen_count = 0
    seen_correct = 0
    unseen_correct = 0
    for pred in predictions:
        if "The answer" in pred["instruction"]:
            ins = pred["instruction"].split("The answer")[0].strip()
        else:
            ins = pred["instruction"]

        if training_instructions is not None:
            if ins in training_instructions:
                seen_count += 1
                if pred["label"] == pred["pred"]:
                    seen_correct += 1
            else:
                unseen_count += 1
                if pred["label"] == pred["pred"]:
                    unseen_correct += 1
        else:
            unseen_count += 1
            if pred["label"] == pred["pred"]:
                unseen_correct += 1
    total_count = (seen_count + unseen_count)
    total_correct = (seen_correct + unseen_correct)
    acc = {
        "seen_accuracy": seen_correct / seen_count if seen_count != 0 else None,
        "unseen_accuracy": unseen_correct / unseen_count if unseen_count != 0 else None,
        "total_accuracy":  total_correct / total_count,

        "total_count": total_count,
        "seen_count": seen_count,
        "unseen_count": unseen_count,

        "total_correct": total_correct,
        "seen_correct": seen_correct,
        "unseen_correct": unseen_correct,

        "unseen_ratio": unseen_count / total_count
    }
    acc = {k: f"{v:.4f}" if v is not None else "" for k,v in acc.items()}
    return acc

In [17]:
from pathlib import Path
import json

# training dataset for getting instruction
train_data_path = Path("/home/u2619111/hank/Dataset/big-superb-train-data-renamed")
task2path = {}
for d in train_data_path.iterdir():
    task2path[d.stem.lower()] = d
    task2path[d.stem.split("_")[0].lower()] = d

# start calculating
exp_path = Path("/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/imagebind_newdata/imagebind1/results3/full")
all_datasets = open("data/test_dataset.txt").read().split("\n")

data_stats = {}
for dataset_name in all_datasets:
    task_data_name = dataset_name
    task_name = task_data_name.split("_")[0].lower()

    row = [task_data_name]
    result_file = (exp_path/f"{task_data_name}.json")
    if (result_file.exists()):
        results = json.load(result_file.open())
        if task2path.get(task_name) or task2path.get(task_data_name.lower()):
            if task2path.get(task_data_name.lower()):
                # in training set
                metadata_file = task2path.get(task_data_name.lower()) / "train/metadata.json"
                metadata = json.load(metadata_file.open())
            else:
                metadata_file = task2path.get(task_name) / "train/metadata.json"
                metadata = json.load(metadata_file.open())

            # gather training instructions and remove label prompts.
            training_instructions = set([v["instruction"].split("The answer")[0].strip() for v in metadata.values()])

            acc = cal_accuracy(results["predictions"], training_instructions)
            row += [acc["total_accuracy"], acc["seen_accuracy"], acc["unseen_accuracy"]]
        else:
            acc = cal_accuracy(results["predictions"])
            row += [acc["total_accuracy"], "", acc["unseen_accuracy"]]
        
        data_stats[task_data_name] = {
            "seen_total": int(float(acc["seen_count"])),
            "unseen_total": int(float(acc["unseen_count"]))
        }
    else:
        row += ["", "", ""]
    print(";".join(row))

BirdSoundDetection_Warblrb10k;0.2829;;0.2829
ChordClassification_AcousticGuitarAndPiano;0.4435;;0.4435
EnvironmentalSoundClassification_AnimalsESC50;0.7375;;0.7375
EnvironmentalSoundClassification_ExteriorAndUrbanNoisesESC50;0.4875;;0.4875
EnvironmentalSoundClassification_HumanAndNonSpeechSoundsESC50;0.1200;;0.1200
EnvironmentalSoundClassification_InteriorAndDomesticSoundsESC50;0.2025;;0.2025
EnvironmentalSoundClassification_NaturalSoundscapesAndWaterSoundsESC50;0.2275;;0.2275
SpeechDetection_LJSpeech;0.9975;0.9973;0.9980
SpeechDetection_LibriSpeechTestClean;0.8973;0.8977;0.8965
SpeechDetection_LibriSpeechTestOther;0.8285;0.8355;0.8125
SpeechTextMatching_LJSpeech;0.5140;0.5114;0.5203
SpeechTextMatching_LibriSpeechTestClean;0.5153;0.5108;0.5261
SpeechTextMatching_LibriSpeechTestOther;0.4974;0.4886;0.5184
SpokenTermDetection_LJSpeech;0.5977;0.6018;0.5878
SpokenTermDetection_LibriSpeechTestClean;0.4607;0.4552;0.4737
SpokenTermDetection_LibriSpeechTestOther;0.4753;0.4670;0.4964
SpeechComma

In [19]:
json.dump(data_stats, open("./instructions_count2.json", "w"), indent=2)

In [18]:
data_stats

{'BirdSoundDetection_Warblrb10k': {'seen_total': 0, 'unseen_total': 8000},
 'ChordClassification_AcousticGuitarAndPiano': {'seen_total': 0,
  'unseen_total': 859},
 'EnvironmentalSoundClassification_AnimalsESC50': {'seen_total': 0,
  'unseen_total': 400},
 'EnvironmentalSoundClassification_ExteriorAndUrbanNoisesESC50': {'seen_total': 0,
  'unseen_total': 400},
 'EnvironmentalSoundClassification_HumanAndNonSpeechSoundsESC50': {'seen_total': 0,
  'unseen_total': 400},
 'EnvironmentalSoundClassification_InteriorAndDomesticSoundsESC50': {'seen_total': 0,
  'unseen_total': 400},
 'EnvironmentalSoundClassification_NaturalSoundscapesAndWaterSoundsESC50': {'seen_total': 0,
  'unseen_total': 400},
 'SpeechDetection_LJSpeech': {'seen_total': 9108, 'unseen_total': 3992},
 'SpeechDetection_LibriSpeechTestClean': {'seen_total': 1818,
  'unseen_total': 802},
 'SpeechDetection_LibriSpeechTestOther': {'seen_total': 2043,
  'unseen_total': 896},
 'SpeechTextMatching_LJSpeech': {'seen_total': 9202, 'uns